In [1]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 検索結果を返す関数の作成
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})

In [3]:
# テスト用コード
ret = get_search_result("東京駅のイベントを教えて")
json.loads(ret)

{'result': [{'url': 'https://www.enjoytokyo.jp/event/list/area1306/',
   'title': "東京駅周辺・丸の内でおすすめのイベント - Let's ENJOY TOKYO",
   'content': '～ 2025/06/01(日) 休館日：月曜日（5月5日は開館）、5月7日 ～ 2025/06/01(日) ～ 2025/06/01(日) 休館日：月曜日（5月5日は開館）、5月7日 ～ 2025/06/03(火) ～ 2025/06/03(火) ～ 2025/06/03(火) 2025/05/29(木) ～ 09/07(日) 伊勢丹 立川店：3/26〜4/15 国際フォーラム：7/14〜7/31 日本橋三越本店：8/6〜8/19 文京グリーンコート：9/1〜9/30 2025/05/21(水) ～ 06/30(月) 熊本が誇る旬の逸品“春スイカ”を首都圏で楽しめる「熊本 春スイカ フェスタ」が開催！ 2025/04/08(火) ～ 07/27(日) 休室日：月曜日、7月8日(火)～13日(日) 2025/05/19(月) ～ 06/08(日) ～ 2025/05/31(土) 2025/06/21(土) ～ 06/22(日) 2025/06/21(土) ～ 06/22(日) 2026/02/07(土) ～ 05/24(日) 休館日：2026/02/16(月)、03/16(月)、04/13(月)、05/11(月) 20世紀北欧デザインの巨匠 スティグ・リンドベリ展 2025/08/20(水) ～ 09/07(日) 20世紀北欧デザインの巨匠 スティグ・リンドベリ展 20世紀北欧デザインの巨匠 スティグ・リンドベリの作品約300点を展示！ 2025/07/26(土) ～ 08/15(金) 2025/06/06(金) ～ 07/06(日) 2025/06/14(土) ～ 07/21(月・祝) 2025/05/31(土) ～ 06/01(日) ～ 2025/05/31(土) 2025/05/13(火) ～ 07/06(日) 休館日は月曜日。開映日時は作品によって異なる。 天空茶会第二十四葉 特別セミナー世界の茶産地を訪ねて セミナー午前の部\u3000ネパールとの出会い 天空茶会第二十四葉 特別セミ

In [4]:
# ツール定義
def define_tools():
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "指定した質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]

In [5]:
# 言語モデルへの質問を行う関数
def ask_question(question, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": question}],
        tools=tools,
        tool_choice="auto",
    )
    return response

In [6]:
# ツール呼び出しが必要な場合の処理を行う関数
def handle_tool_call(response, question):
    # 関数の実行と結果取得
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)

    # 関数の実行結果をmessagesに加えて再度言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )
    return response_after_tool_call

In [7]:
# ユーザーからの質問を処理する関数
def process_response(question, tools):
    response = ask_question(question, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        final_response = handle_tool_call(response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()

In [8]:
tools = define_tools()

# 言語モデルが直接回答できる質問
question = "東京都と沖縄県はどちらが広いですか？"
response_message = process_response(question, tools)
print(response_message)

東京都の面積は約2,194 km²で、沖縄県の面積は約2,281 km²です。したがって、沖縄県の方が東京都よりも広いということになります。


In [9]:
tools = define_tools()

# ツール呼出が必要な質問
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"
response_message = process_response(question, tools)
print(response_message)

最近の東京駅周辺でのイベント情報は以下の通りです。

1. **Let's ENJOY TOKYO**
   - **日時**: 2025年5月28日（水）〜6月30日（月）
   - **場所**: XEX TOKYO（東京駅直結の大丸東京上）
   - **内容**: 様々な食のイベントや特集が行われる。

   詳細は[こちら](https://www.enjoytokyo.jp/event/list/sta200101/)。

2. **東京駅「キラキラ」イベント**
   - **日時**: 2025年4月10日（木）〜6月30日（月）
   - **内容**: 年末の盛り上がりを感じさせる特別なライトアップやアートの展示が行われています。

   詳細は[こちら](https://www.tokyo-np.co.jp/article/76521)。

3. **東京駅イベント情報**
   - **日時**: 2025年5月29日（木）〜9月7日（日）
   - **内容**: 地元の特産品をフィーチャーしたフェアや市などが行われる予定。

   詳細は[こちら](https://www.tokyoeki-1bangai.co.jp/event/)。

これらのイベントは、東京駅周辺で楽しむことができるアクティビティや特別な体験を提供しています。イベントに関するさらなる情報は、各リンク先をご覧ください。


In [10]:
# チャットボットへの組み込み
tools = define_tools()

messages=[]

while(True):
    # ユーザーからの質問を受付
    question = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if question.strip()=="":
        break
    display(f"質問:{question}")

    # メッセージにユーザーからの質問を追加
    messages.append({"role": "user", "content": question.strip()})
    # やりとりが8を超えたら古いメッセージから削除
    if len(messages) > 8:
        del_message = messages.pop(0)

    # 言語モデルに質問
    response_message = process_response(question, tools)

    # メッセージに言語モデルからの回答を追加
    print(response_message, flush=True)
    messages.append({"role": "assistant", "content": response_message})

print("\n---ご利用ありがとうございました！---")

'質問:こんにちは！'

こんにちは！今日はどんなことをお手伝いできますか？


'質問:東北6県は？'

東北地方は日本の北東部に位置し、以下の6つの県から成り立っています：

1. **青森県**（あおもりけん）
2. **岩手県**（いわてけん）
3. **宮城県**（みやぎけん）
4. **秋田県**（あきたけん）
5. **山形県**（やまがたけん）
6. **福島県**（ふくしまけん）

これらの県は、自然豊かで歴史や文化も深く、多くの観光地や特色があります。


'質問:宮城県のお土産について検索した結果を教えて'

宮城県で人気のお土産についての情報をいくつか紹介します。

1. **仙台名物の『萩の月』**:
   - 『萩の月』は、カステラ生地の中にクリームが入った和菓子で、宮城県の代表的なお土産です。

2. **ずんだ餅**:
   - ずんだ餅は、枝豆をすりつぶしたものを使った餅で、独特の甘さが人気です。仙台のお土産として広く知られています。

3. **仙台牛タン**:
   - 仙台名物の牛タンは、厚切りで焼き上げたものが特徴です。お土産用には真空パックされたものも多く販売されています。

4. **まりんがの鯖（好き造りされたもの）**:
   - 賞味期限が比較的短いですが、新鮮さが際立つ鯖は、特に海の近くで人気のあるお土産です。

5. **『こぶ茶』**:
   - こぶ茶は、昆布を用いたお茶で、健康志向の人々に支持されています。

6. **仙台のお菓子『喜久福』**:
   - モチ米を使った和菓子で、クリームやあんこが詰められているものがあります。

### 参考リンク
- [仙台の名産お土産に関する詳細記事](https://tohokuru.jp/blogs/feature/miyage_miyagi) 
- [宮城県のお土産20選](https://tsplus.asahi.co.jp/articles/gift/75865/)

これらの情報を参考に、宮城県を訪れた際にはぜひお土産を選んでみてください！

---ご利用ありがとうございました！---
